In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import re
import os
import ast
import cv2
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
import folium
import pickle
from folium.plugins import HeatMap
from mpl_toolkits.mplot3d import Axes3D
from datetime import date
from scipy import stats
from scipy.stats import norm,skew
from scipy.special import boxcox1p,inv_boxcox1p,boxcox
from sklearn.metrics import mean_squared_log_error, accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder,StandardScaler
from sklearn.base import BaseEstimator,TransformerMixin,RegressorMixin,clone
from collections import defaultdict
import lightgbm as lgb
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 5);
sns.set_style('whitegrid')

# CONSTANTS

In [ ]:
RESTAURANT_NUMBER = 85
LIKE = 2

# DATASET

customer_id 86
gender
status_x
verified_x
location_number 90
location_type
latitude_x
longitude_x
id
latitude_y 95
longitude_y
vendor_category_id
delivery_charge
serving_distance
prepration_time 100
discount_percentage
rank
vendor_rating
device_type
location_number_obj 105
id_obj
target 107

In [ ]:
train_data = []
val_data = []
test_data = []
train_labels = []
val_labels = []
test_labels = []
for i in range(RESTAURANT_NUMBER):
  #107 target 
  cols = [0,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106]
  #[0,86,90, 91, 92, 93, 94, 95, 96]
  
  data = pd.read_csv("/content/drive/MyDrive/Bil573/train_data/restaurant_" + str(i) + ".csv")

  data.drop(data.columns[cols], axis=1, inplace=True)
  
  df1 = data.iloc[:,:-1]
  df1_labels = data[data.columns[-1]]
  
  data = pd.read_csv("/content/drive/MyDrive/Bil573/validation_data/restaurant_" + str(i) + ".csv")
  data.drop(data.columns[cols], axis=1, inplace=True)

  df2 = data.iloc[:,:-1]
  df2_labels = data[data.columns[-1]]         
  frames = [df1,df2]     
  df = pd.concat([df1,df2])
  frames = [df1_labels,df2_labels]
  df_labels =  pd.concat([df1_labels,df2_labels])

  train_data.append(df)
  train_labels.append(df_labels)
  data = pd.read_csv("/content/drive/MyDrive/Bil573/test_data/restaurant_" + str(i) + ".csv")
  #Customer id yi(86 no) burada drop etmeeee
  cols = [0,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106]
  #[0,90, 91, 92, 93, 94, 95, 96]
  
  data.drop(data.columns[cols], axis=1, inplace=True)
  test_data.append(data.iloc[:,:-1])
  test_labels.append(data[data.columns[-1]])

In [ ]:
model_path = "/content/drive/MyDrive/Bil573/SVM/svm_model"

In [ ]:
model_path = "/content/drive/MyDrive/Bil573/LGBM/lgbm_model"

# TRAINING

In [ ]:
models = []
for m in range (RESTAURANT_NUMBER-1):
  auc_best = 0

  model = LGBMClassifier(scoring='roc_auc')
 # model = make_pipeline(StandardScaler(), SVC(gamma='auto'))
  model.fit(train_data[m], train_labels[m])

  models.append(model)
  pickle.dump(model, open(model_path + str(m) + ".pkl", 'wb'))##
      

# TEST

In [ ]:
preds = []
labels = []
ids = []
for m in range (RESTAURANT_NUMBER-1):
  model = pickle.load(open(model_path + str(m) + ".pkl", 'rb'))
  data = test_data[m].copy()
  
  data.drop(["customer_id"], axis=1, inplace=True)
  pred = model.predict(data)

  data = test_data[m].copy()
  preds = np.concatenate((preds, pred), axis=0)
  labels = np.concatenate((labels, test_labels[m]), axis=0)
 
  ids = np.concatenate((ids, data[data.columns[RESTAURANT_NUMBER]]), axis=0)
 
auc = roc_auc_score(labels, preds)
auc#ROC score

0.5423395907316488

In [ ]:
data.columns[RESTAURANT_NUMBER]

'customer_id'

In [ ]:
results = np.zeros((len(ids), 3))
results[:, 0] = ids
results[:, 1] = labels
results[:, 2] = preds
columns = ['id','label', 'pred']
results = pd.DataFrame(data=results, columns=columns)

In [ ]:
accs = []
accs2 = []
labels2 = []
accs3 = []
preds3 = []
for i in range(35000):
  customer = results[results['id']==i]
  size = customer.shape
  if size[0] != 0:
    result = customer[customer.columns[1:]]
    acc = accuracy_score(result['label'], result['pred'])
    accs.append(acc)

    res = result[result['pred']==1.]
    size = res.shape
    if size[0] != 0:
      labels2.append(res['label'])
      acc = accuracy_score(res['label'], res['pred'])
      accs2.append(acc)
    
    res = result[result['label']==1.]
    size = res.shape
    if size[0] != 0:
      preds3.append(res['pred'])
      acc = accuracy_score(res['label'], res['pred'])
      accs3.append(acc)

sum(accs)/len(accs)#Total accuracy

0.8777282642142687

In [ ]:
len(accs)

2855

In [ ]:
sum(accs2)/len(accs2)#önerilenlerin(+) accuracy 

0.5799562971791816

In [ ]:
hit_customer = 0
for i in range(len(labels2)):
  if sum(labels2[i]) > 0:
    hit_customer += 1

hit_customer/len(accs2)
# insan önerdiklerimden en az 1 ini beğendi mi

0.6293206197854588

In [ ]:
len(accs2)#

839

In [ ]:
hit_customer = 0
for i in range(len(preds3)):
  if sum(preds3[i]) > 0:
    hit_customer += 1

hit_customer/len(accs3)
# insanın beğenebileceklerinden en az 1 ini önerdim mi

0.20480993017843288

In [ ]:
len(accs3)

2578

In [ ]:
# KAÇ KİŞİNİN BEĞENDİĞİ YER VAR/YOK

In [ ]:
# BEĞENİLEN YERLERDEN KAÇINI ISKALADI